# 긍정 / 부정 댓글 분석

## 1. 라이브러리 추가

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

## 2. 데이터 불러오기 및 전처리

In [3]:
train_df = pd.read_csv('../data_list/train_df.csv', encoding='utf-8')
train_df[:3]

,review_score,review_content,label
0,5,배송 빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주 좋아요 바지 정말 좋아서개 더 구매했어요 이 가격에 대박입니다 바느질이 조금 ...,1


In [4]:
train_df = train_df[train_df['review_content'].notnull()]
train_df['review_content'] = train_df['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))

In [5]:
test_df = pd.read_csv('../data_list/test_df.csv', encoding='utf-8')
test_df[:3]

,review_score,review_content,label
0,4.5,색감이 곱네요,1.0
1,5.0,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,1.0
2,4.5,우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...,0.0


In [6]:
test_df = test_df[test_df['review_content'].notnull()]
test_df['review_content'] = test_df['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))

In [7]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])
stopwords[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']

## 3. tf-idf --> fit_transform

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

In [9]:
okt = Okt()

In [10]:
tfidf = TfidfVectorizer(tokenizer = okt.morphs,
                        ngram_range=(1, 2),
                        min_df = 3,
                        max_df = 0.9,
                        stop_words = stopwords)
tfidf

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...],
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x00000210FBAF44E0>>)

In [11]:
tfidf.fit(train_df['review_content'])

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...],
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x00000210FBAF44E0>>)

In [12]:
train_tfidf_df = tfidf.transform(train_df['review_content'])
train_tfidf_df

<40x24 sparse matrix of type '<class 'numpy.float64'>'
	with 101 stored elements in Compressed Sparse Row format>

In [13]:
test_tfidf_df = tfidf.transform(test_df['review_content'])
test_tfidf_df

<11x24 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [14]:
tfidf.inverse_transform(train_tfidf_df)

[array(['배송'], dtype='<U4'),
 array(['용', '도'], dtype='<U4'),
 array(['좋아요', '입니다', '구매', '개'], dtype='<U4'),
 array(['하는', '용', '안', '아쉽네요', '생각', '배송', '도'], dtype='<U4'),
 array(['용', '사용', '도'], dtype='<U4'),
 array(['입니다', '생각', '보다', '별로', '도', 'ㅠㅠ'], dtype='<U4'),
 array(['은', '배송', '도'], dtype='<U4'),
 array(['안', '별로'], dtype='<U4'),
 array(['은', '요'], dtype='<U4'),
 array(['보다', '도', '다'], dtype='<U4'),
 array(['좋아요', '도', '넘'], dtype='<U4'),
 array(['구매'], dtype='<U4'),
 array(['그냥'], dtype='<U4'),
 array([], dtype='<U4'),
 array(['좋고', '도'], dtype='<U4'),
 array(['좋아요', '좋고', '도', '너무'], dtype='<U4'),
 array(['재구매', '요'], dtype='<U4'),
 array([], dtype='<U4'),
 array(['재구매', '다'], dtype='<U4'),
 array(['하는', '구매'], dtype='<U4'),
 array(['개'], dtype='<U4'),
 array(['요', '별로', '고'], dtype='<U4'),
 array(['좋아요', '도', '너무'], dtype='<U4'),
 array(['은'], dtype='<U4'),
 array(['재구매', '배송'], dtype='<U4'),
 array(['좋고', '고'], dtype='<U4'),
 array(['하는', '은', '요', '배송'], dtype='<U4')

In [15]:
tfidf.inverse_transform(test_tfidf_df)

[array([], dtype='<U4'),
 array([], dtype='<U4'),
 array(['은', '너무'], dtype='<U4'),
 array(['도', '너무'], dtype='<U4'),
 array(['은', '요', '생각', '보다', '그냥'], dtype='<U4'),
 array(['하는', '그냥'], dtype='<U4'),
 array(['은', '생각', '배송'], dtype='<U4'),
 array(['생각', '보다'], dtype='<U4'),
 array(['생각', '보다'], dtype='<U4'),
 array(['도'], dtype='<U4'),
 array(['별로'], dtype='<U4')]

## 4. 분석 정확도 확인하기

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
SA_lr = LogisticRegression(random_state = 0)

In [18]:
SA_lr.fit(train_tfidf_df, train_df['label'])

LogisticRegression(random_state=0)

In [19]:
test_predict = SA_lr.predict(test_tfidf_df)

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
print('후기 분석(긍정/부정) 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

후기 분석(긍정/부정) 정확도 :  0.727


## 5. 후기 긍정/부정 분석하기

In [22]:
st = test_df['review_content'][9]

In [23]:
st2 = re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", st)
st2

'옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여'

In [24]:
st_tfidf_df = tfidf.transform([st2])
st_tfidf_df

<1x24 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [25]:
st2_predict = SA_lr.predict(st_tfidf_df)
st2_predict, st2_predict[0]

(array([1], dtype=int64), 1)

In [26]:
if(st2_predict[0] == 0):
    print(st2, '==> 부정 감정')
else:
    print(st2, '==> 긍정 감정')

옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여 ==> 긍정 감정


In [27]:
st3 = "색이 사진이랑 다르게 벽돌색이네요 별로에요.. 다시는 구매하고싶지 않아요!! 사기도 정도껏 치세요.."

In [28]:
st4 = re.sub(r'[^ ㄱ-ㅎ|가-힣]+', "", st3)
st4

'색이 사진이랑 다르게 벽돌색이네요 별로에요 다시는 구매하고싶지 않아요 사기도 정도껏 치세요'

In [29]:
st_tfidf_df2 = tfidf.transform([st4])
st_tfidf_df2

<1x24 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [30]:
st4_predict = SA_lr.predict(st_tfidf_df2)
st4_predict, st2_predict[0]

(array([0], dtype=int64), 1)

In [31]:
if(st4_predict[0] == 0):
    print(st4, '==> 부정 감정')
else:
    print(st4, '==> 긍정 감정')

색이 사진이랑 다르게 벽돌색이네요 별로에요 다시는 구매하고싶지 않아요 사기도 정도껏 치세요 ==> 부정 감정


## 6. review 긍정 갯수 / 부정 갯수 조회

In [32]:
review = test_df.copy()
review[:3]

,review_score,review_content,label
0,4.5,색감이 곱네요,1.0
1,5.0,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,1.0
2,4.5,우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...,0.0


In [33]:
pos_review = review[review['label'] == 1]
neg_review = review[review['label'] == 0]

In [34]:
print('긍정적인 후기는 총', len(pos_review), '개입니다.')
print('부정적인 후기는 총', len(neg_review), '개입니다.')

긍정적인 후기는 총 8 개입니다.
부정적인 후기는 총 3 개입니다.
